In [ ]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
import sys
# reload(sys)
# sys.setdefaultencoding('utf8')
import os
import json
import string
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import random
from numpy import array
import re
import pickle
#from pattern.text.en import singularize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#import enchant
from nltk.stem import LancasterStemmer
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import operator
from collections import Counter
from nltk.corpus import wordnet as wn
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from collections import defaultdict
import math
import csv

import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import gzip

import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string
from nltk.corpus import stopwords
import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex


import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable

import spacy
import string

from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.corpus import stopwords

torch.manual_seed(1)


In [ ]:
# ##load the predicted data
# predicted_df = pd.read_csv( 'data/keto/predicted_keto_14k.csv',low_memory=False) #(28189, 6)
# #print(predicted_df.shape) ## (14320, 7)
# #predicted_df

In [ ]:
# ###Baseline
# df = predicted_df[['name', 'description', 'text', 'utype', 'utype_gt']] #extracted 5 columns

In [ ]:
# #take the rows where weak label (utype) is not NAN. We will use this to train our supervised model
# df_train = df[df['utype'].notna()] #858 # train data

# df_test = df.drop(df_train.index) #13462 # test data (no weak label)


In [ ]:
# df_train = df_train.reset_index(drop=True) 

# df_test = df_test.reset_index(drop=True) 

# df_trn = df_train.sample(frac = 0.80) #686
# # rest of the 20% values
# df_val = df_train.drop(df_trn.index) #421
# df_trn = df_trn.reset_index(drop=True) 

# df_val = df_val.reset_index(drop=True) #172

In [ ]:
# df_trn.to_csv('data/keto/baseline_train_keto_weak_cv3.csv', line_terminator='\r\n', encoding='utf-8', index = False) 
# df_val.to_csv('data/keto/baseline_val_keto_weak_cv3.csv', line_terminator='\r\n', encoding='utf-8', index = False) 
# df_test.to_csv('data/keto/baseline_test_keto_weak_cv3.csv', line_terminator='\r\n', encoding='utf-8', index = False)

In [ ]:
df_trn = pd.read_csv( 'data/keto/baseline_train_keto_weak_cv3.csv',low_memory=False)

In [ ]:
#tokenization
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [ ]:
#count number of occurences of each word
counts = Counter()
for index, row in df_trn.iterrows():
    counts.update(tokenize(row['text']))

In [ ]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

In [ ]:
#creating vocabulary on train data
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
def encode_sentence(text, vocab2index, N=300):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [ ]:
df_trn['encoded'] = df_trn['text'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
#df_trn.head()

In [ ]:
df_val = pd.read_csv( 'data/keto/baseline_val_keto_weak_cv3.csv',low_memory=False)

In [ ]:
df_val['encoded'] = df_val['text'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
#df_val.head()

In [ ]:
X_train = list(df_trn['encoded'])
y_train = list(df_trn['utype'])


X_valid = list(df_val['encoded'])
y_valid = list(df_val['utype'])


In [ ]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [ ]:
class Keto(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [ ]:
train_ds = Keto(X_train, y_train)
valid_ds = Keto(X_valid, y_valid)

In [ ]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    best_valid_loss = float('inf')
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if val_loss <= best_valid_loss:
            best_valid_loss = val_loss
            print("best model saved in epoch :", i+1 )
            torch.save(model.state_dict(), 'data/keto/weak_keto_model_cv3_'+str(i+1)+'.pt')
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [ ]:
batch_size = 16
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
def load_glove_vectors(glove_file="data/glove.6B.300d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [ ]:
def get_emb_matrix(pretrained, word_counts, emb_size = 300):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [ ]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

In [ ]:
class LSTM_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x,l):
        #print(x.size)
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
print(vocab_size) 

In [ ]:
model = LSTM_glove_vecs(vocab_size, 300, 150, pretrained_weights)

In [ ]:
train_model(model, epochs=20, lr=0.01)

In [ ]:
#for test prediction
df_test = pd.read_csv( 'data/keto/baseline_test_keto_weak_cv3.csv',low_memory=False)

In [ ]:
df_test['encoded'] = df_test['text'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
#df_test.head()

In [ ]:
X_test = list(df_test['encoded'])
y_test = list(df_test['utype'])

In [ ]:
test_ds = Keto(X_test, y_test)

In [ ]:
test_dl = DataLoader(test_ds, batch_size=1)

In [ ]:
def test_prediction (model, test_dl):
    y_pred = []
    model.eval()
    for x, y, l in test_dl:
        x = x.long()
        y = y.long()
        #print(l)
        y_hat = model(x, l)
        pred = torch.max(y_hat, 1)[1]
        y_pred.append(pred.item())
    return y_pred
        

In [ ]:
model.load_state_dict(torch.load('data/keto/weak_keto_model_cv3_3.pt')) 
test_pred = test_prediction(model, test_dl)

In [ ]:
df_test['utype_pred'] = test_pred

In [ ]:
##Baseline (Glove + LSTM) -- accuracy
df_b = df_test[['name', 'utype_pred', 'utype_gt']] #extracted 3 columns
#print(df_b)
df_b1= df_b.dropna()#dropping nan value
#print(df_b1) #680
df_b1 = df_b1[df_b1.utype_gt != 2.0] #dropping rows with truth value 2.0
df_b1 = df_b1.reset_index(drop = True)
#print(df_b1) #335
count_label = 0

for i in range (0, df_b1.shape[0]):
    if df_b1.utype_pred[i] == df_b1.utype_gt[i]: #count label those are same for both ground truth and prediction
        count_label = count_label + 1
        
print('count_label', count_label) #136
print('accuracy of baseline : ', (count_label/df_b1.shape[0]) ) 
# cv1 = 0.72, cv2 = 0.72, cv3 = 0.71
#mean = 0.72

##Baseline (Glove + LSTM) -- Macro avg F1 score
from sklearn.metrics import f1_score
print('Macro-avg F1 score of baseline : ', f1_score(df_b1.utype_gt.values, df_b1.utype_pred.values, average='macro')) 

#cv1= 0.42, cv2 = 0.42, cv3 = 0.44

#mean = 0.43


